# MMVCの学習

ver.2023/3/9

MMVCで利用するVITSの学習を行います。

​

In [ ]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行してください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown セルの実行が完了したら、次に進んでください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
jst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#git-lfs 事前学習済みモデルのダウンロードに使用
!apt install git-lfs
#os ディレクトリの存在確認に使用
import os
#sys Pythonの管理全般(クラッシュ、tracebackなど)
import sys
print("log: パッケージのインストールが完了しました。\n")


​

In [ ]:
%%ccapture one_prepare_notebook
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
%%ccapture two_prepare_repo
#@title ## 2 リポジトリの準備
#@markdown リポジトリの準備を行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。

#@markdown 設定が完了したら、次へを押してください。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

#---関数終わり---

def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  global path_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  path_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', placeholder='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', description='Path:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, path_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global platform #グローバル指定
  platform = platform_input.value
  global path #グローバル指定
  path = path_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    %cd $path
  elif platform == "Local":
    %cd $path

#実行
first_view()

​

In [ ]:
%%ccapture three_prepare_training
#@title ## 3 実行の準備
#@markdown 実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

print("log: パッケージのインストールを開始します。")
!apt-get install espeak
!pip install -r requirements.txt
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
%%ccapture four_check_gpu
#@title ## 4 GPUの確認
#@markdown GPUの確認を行います。
#@markdown 割り当てられたGPUのメモリーを確認し、必要に応じてconfigファイルの"batch_size"を変更してください。 
#@markdown あくまで参考値ですが、GPUメモリ40GBで"batch_size"：16で動きました。 

#@markdown よく分からない場合、変更不要です。(学習においてメモリの不足によりエラーとなる場合、変更が必要となります。詳細についてはDiscordでお問い合わせください)
!nvidia-smi

​

In [ ]:
%%ccapture five_launch_tensorboard
#@title ## 5 tensorboardの起動
#@markdown とりあえず実行しておいてください。

#Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs --port 5000

​

In [ ]:
%%ccapture six_settings_for_training
#@title ## 6 学習の設定
#@markdown 学習に必要な設定を行います。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown New / Resume
#@markdown 新規に学習を開始する場合は、Newを選択し、**以下の-c, -m, -fg, -fdを全て設定してください。**
#@markdown 学習を再開する場合は、Resumeを選択し、以下の-c, -mを設定してください。-mで指定したディレクトリの最新のモデルから学習を再開します。

#@markdown -c：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。

#@markdown -m：modelの保存先ディレクトリ  
#@markdown **ディレクトリ名を直接指定してください。** 

#@markdown -f：(新規学習時のみ) Fine tuningのベースとなるG/D_xxxx.pth のファイル名を指定してください。 
#@markdown latestを指定すると、自動で最新のファイルを選択します。 
#@markdown よく分からない場合は、変更不要です。

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。
#@markdown 「設定が完了しました」と表示されたら、次のセルに進んでください。

def first_view():
  #widgetsの構成
  global new_or_resume_input #グローバル指定
  global config_path_input #グローバル指定
  global model_save_dic_input #グローバル指定
  global fine_model_input #グローバル指定
  new_or_resume_input = widgets.Dropdown(options=["New", "Resume"], value="New", description='new/resume:', disabled=False)
  config_path_input = widgets.Text(value='configs/train_config.json', placeholder='configs/train_config.json', description='config_path:', disabled=False)
  model_save_dic_input = widgets.Text(value='tutorial', placeholder='tutorial', description='model_save_dic:', disabled=False)
  fine_model_input = widgets.Text(value='latest', placeholder='G/D_xxxxx.pth', description='fine_model:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(new_or_resume_input, config_path_input, model_save_dic_input, fine_model_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global new_or_resume #グローバル指定
  new_or_resume = new_or_resume_input.value
  global config_path #グローバル指定
  config_path = config_path_input.value
  global model_save_dic #グローバル指定
  model_save_dic = model_save_dic_input.value
  #fine_model_gとfine_model_dは新規のみ
  if new_or_resume == "New":
    global fine_model_g #グローバル指定
    if fine_model_input.value == "latest":
      filenames = [f for f in os.listdir(path + "/fine_model") if f.startswith('G_v') and f.endswith('_finemodel.pth')]
      filenames = sorted(filenames, key=lambda x: int(x.split('_')[1][2:3]) * 100 + int(x.split('_')[2][4:5]))
      if filenames:
        fine_model_g = "fine_model/" + filenames[-1]
      else:
        print("finemodelが見つかりません。")
    else:
      fine_model_g = "fine_model/" + fine_model_input.value
    global fine_model_d #グローバル指定
    if fine_model_g:
      fine_model_d = fine_model_g.replace('G', 'D')
    else:
      print("finemodelが見つかりません。")
  %cd {path}
  print("設定が完了しました。")
  print("new_or_resume:" + new_or_resume)
  print("config:" + config_path)
  print("model_save_dic:" + model_save_dic)
  if new_or_resume == "New":
    print("fine_model_g:" + fine_model_g)
    print("fine_model_d:" + fine_model_d)

#実行
first_view()

​

In [ ]:
#@title ## 7 学習を行う
#@markdown 学習を行います。

#@markdown セルを実行すると、学習が開始されます。
#@markdown 学習はインスタンスが切断されるか、何らかのエラーが発生するまで継続します。

#@markdown 学習を終了するタイミングについては、Discordでお問い合わせください。

if new_or_resume == "New":
  !python train_ms.py -c $config_path -m $model_save_dic -fg $fine_model_g -fd $fine_model_d
elif new_or_resume == "Resume":
  !python train_ms.py -c $config_path -m $model_save_dic

In [ ]:
%%ccapture eight_convert_to_onnx
print("----------------------------------------------------------------------------------------------------")
print("8 学習結果をONNXファイルに変換")

#@title ## 8 学習結果をONNXファイルに変換

#@markdown 学習結果をONNXファイルに変換します。 

#@markdown ONNXファイルに変換するモデルを選択してください。

#@markdown (基本的にvc_conf_passは設定変更不要です)
model_save_dic = "tutorial" #@param {type:"string"}
model_pass = "G_latest_99999999.pth" #@param {type:"string"}
vc_conf_pass = "configs/myprofile.conf" #@param {type:"string"}

!python onnx_export.py --config_file logs/$model_save_dic/config.json --client_conf configs/myprofile.conf --convert_pth logs/$model_save_dic/$model_pass


#@markdown 変換が終わるとlogs内にあるmodelの保存先ディレクトリにONNXファイルが生成されています。  

#@markdown ONNXを使ったクライアントの実行には、このディレクトリ内の **config.json** と **G_xxxx.onnx** の2ファイルが必要です。  

#@markdown クライアントのlogsディレクトリ内に、わかりやすいように同じ名前でフォルダを作成し、この2ファイルをコピーしてください。

#@markdown (v1.3.1.x系のように **config.json** と **G_xxxx.pth** でもクライアントは実行できますが、onnxを使った方が高速です。)

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル
#@markdown * version.txt (MMVCのバージョンが記載されているファイル)

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
version_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-version_txt.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {path} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#version.txtの保存
try:
  !cp {path}/version.txt {version_txt}
except:
  pass

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {path}/logs ]; then if [ -z "$(ls {path}/logs)" ]; then touch {path}/logs/no-file.txt;else find . -name "*.log" -exec cp {} ~/mmvc-debug/logs \;; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {path}/configs ]; then if [ -z "$(ls {path}/configs)" ]; then touch {path}/configs/no-file.txt;else cp -rp {path}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  try:
    print(sys.last_type, sys.last_value, sys.last_traceback, file=f)
  except:
    pass

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/one_prepare_notebook.txt', 'w') as f:
  try:
    print(one_prepare_notebook, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/two_prepare_repo.txt', 'w') as f:
  try:
    print(two_prepare_repo, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/three_prepare_training.txt', 'w') as f:
  try:
    print(three_prepare_training, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/four_check_gpu.txt', 'w') as f:
  try:
    print(four_check_gpu, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/five_launch_tensorboard.txt', 'w') as f:
  try:
    print(five_launch_tensorboard, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/six_settings_for_training.txt', 'w') as f:
  try:
    print(six_settings_for_training, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/eight_convert_to_onnx.txt', 'w') as f:
  try:
    print(eight_convert_to_onnx, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)